In [10]:
import cv2
import numpy as np
import glob
import seaborn as sns
import math
from sklearn.ensemble import IsolationForest
from matplotlib import pyplot as plt

In [2]:
SIFT_method = True

Questa funzione prende in ingresso l'immagine originale, l'immagine ritagliata e il nome del salvataggio. Restituisce un immagine che rappresenta la zona di lavoro

In [11]:
def cutWorkSpace(img,border): 
    # dimensioni immagini
    img_h = 2048
    img_l = 2048
    # offset per posizionamento su area d'indagine
    offset_x = 600
    offset_y = 500
    
    #taglio per garantire la ricerca sull'area di interesse, evitando rumori dati dallo sfondo
    crop_img = img[offset_y:img_h-offset_y, offset_x:img_l-offset_x]
    
    # Convert the img to grayscale
    gray = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    high_thresh, thresh_im = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #cv2.imwrite('provaT.jpg', thresh_im)
    lowThresh = 0.5*high_thresh
    
    
    # Apply edge detection method on the image
    #edges = cv2.Canny(blurred,20,150,apertureSize = 3)
    edges = cv2.Canny(blurred,20,150,apertureSize = 3)
    #cv2.imwrite('results_img/'+str(count_img)+'ed.jpg', edges)
    #cv2.imwrite('canny.jpg', edges)
    # This returns an array of r and theta values
    lines = cv2.HoughLines(edges,1,np.pi/180, 100)
    #cv2.imwrite('Risultati/resFS'+name+'c.jpg', edges)
    # The below for loop runs till r and theta values 
    # are in the range of the 2d array
    i = 0
    x_first = 0
    x_second = 0
    count = 0
    delta = 30
    thetaFinal = 0
    while count < 2:
        for r,theta in lines[i]:
            # Stores the value of cos(theta) in a
            a = np.cos(theta)

            # Stores the value of sin(theta) in b
            b = np.sin(theta)

            # x0 stores the value rcos(theta)
            x0 = a*r

            # y0 stores the value rsin(theta)
            y0 = b*r

            #ti define line lenght
            const = 3000
            # x1 stores the rounded off value of (rcos(theta)-1000sin(theta))
            x1 = int(x0 + const*(-b))

            # y1 stores the rounded off value of (rsin(theta)+1000cos(theta))
            y1 = int(y0 + const*(a))

            # x2 stores the rounded off value of (rcos(theta)+1000sin(theta))
            x2 = int(x0 - const*(-b))

            # y2 stores the rounded off value of (rsin(theta)-1000cos(theta))
            y2 = int(y0 - const*(a))

            if count == 0 :
                if theta != 0:
                    thetaFinal = 180-math.degrees(theta)
                x_first = x0
                #cv2.line(img,(x1+offset_x,y1+offset_y), (x2+offset_x,y2+offset_y), (0,0,255),2)
                count += 1
            else:
                if abs(x_first-x0) > delta :
                    x_second = x0
                    #cv2.line(img,(x1+offset_x,y1+offset_y), (x2+offset_x,y2+offset_y), (0,0,255),2)
                    count += 1
            i += 1
    # All the changes made in the input image are finally
    # written on a new image houghlines.jpg
    #cv2.imwrite('Risultati/res'+name+'.jpg', img)
    (h, w) = img.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # rotate our image by Theta degrees around the center of the image
    M = cv2.getRotationMatrix2D((cX, cY), -thetaFinal, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    scale_percent = 100 # percent of original size

  
    if x_first < x_second:
        
        median = (x_second+x_first)//2
        temp = rotated[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]
        width = int(temp.shape[1] * scale_percent / 100)
        height = int(temp.shape[0] * scale_percent / 100)
        dim = (width, height)
        temp = cv2.resize(temp, dim, interpolation = cv2.INTER_AREA)        
        gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        return  gray
    else:
        median = (x_first+x_second)//2.
        temp = rotated[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]        
        #blurred = cv2.GaussianBlur(temp, (3,3), 0)
        width = int(temp.shape[1] * scale_percent / 100)
        height = int(temp.shape[0] * scale_percent / 100)
        dim = (width, height)
        temp = cv2.resize(temp, dim, interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        return  gray
    

In [12]:
def divideImage(img,pcs):
    results = []
    lung = len(img) // pcs
    for i in range(pcs):
        results.append(img[(lung*i):(lung*(i+1)-1)][:])

    return results

## Creazione histogramma usando features - creazione NN
accetta in ingresso un vettore con [[img],[classe]]
in uscita mi ritorna kmeans e MLP

In [13]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier

def createNN(imgsWithClass, Nclusters):
    if SIFT_method:
        sift = cv2.xfeatures2d.SIFT_create(nfeatures)
    else:
        orb = cv2.ORB_create()   
    results = imgsWithClass[0]

    keypoints = []
    descriptors = []
    dico = []
    for img in results:
        if SIFT_method:
            keypoints_temp, descriptors_temp = sift.detectAndCompute(img, None)
        else:    
            keypoints_temp, descriptors_temp = orb.detectAndCompute(img, None)
        keypoints.append(keypoints_temp)
        descriptors.append(descriptors_temp)
        for d in descriptors_temp:
            dico.append(d)        
    k = Nclusters
    batch_size = 256*4
    print(k)
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
    
    length = len(results)

    train_index = length - 1 
    train_imgs = results[:train_index]
    validation_imgs = results[train_index:]
    
    # creo histogramma con valori trovati di feature per ogni immagine di train
    kmeans.verbose = False
    histo_list = []

    for img in train_imgs:
        if SIFT_method:
            kp, des = sift.detectAndCompute(img, None)
        else:
            kp, des = orb.detectAndCompute(img,None)
        histo = np.zeros(k)
        nkp = np.size(kp)

        for d in des:
            idx = kmeans.predict([d])
            histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

        histo_list.append(histo)
    #creo X_train e Y_train
    X_train = np.array(histo_list)  
    Y_train = imgsWithClass[1][:train_index]
    
    
    return histo_list,kmeans
    #return kmeans, mlp, val_result

Creo funzione per testare immagini sulla NN trovata

In [53]:
def testNN(img_test,border,pcs,Nclusters):
    img = cutWorkSpace(img_test,border)
    imgs = divideImage(img,pcs)
    if SIFT_method:
        sift = cv2.xfeatures2d.SIFT_create(nfeatures)
    else:
        orb = cv2.ORB_create()   

    k = Nclusters
    results = []
    for img in imgs:
        if SIFT_method:
            kp, des = sift.detectAndCompute(img, None)
        else:
            kp, des = orb.detectAndCompute(img,None)
            
        x = np.zeros(k)
        nkp = np.size(kp)
        for d in des:
            idx = kmeans.predict([d])
            x[idx] += 1/nkp #ottengo vettore normalizzato da kmeans
        t = []
        t.append(x)
        results.append(model.predict(t))

        
    return results

In [32]:
scelta = 120

if scelta == 70:
    pathBuoni = '/Users/leonardoaggio/Desktop/Dataset/S606C/Buoni70/*.tif'
    pathBuoniVal = '/Users/leonardoaggio/Desktop/Dataset/S606C/Buoni70Val/*.tif'
    contamination_N = 6/79.

if scelta == 100:
    pathBuoni = '/Users/leonardoaggio/Desktop/Dataset/S606C/Buoni100/*.tif'
    pathBuoniVal = '/Users/leonardoaggio/Desktop/Dataset/S606C/Buoni100Val/*.tif'
    contamination_N = 6/107.
if scelta == 120:
    pathBuoni = '/Users/leonardoaggio/Desktop/Dataset/S606C/Buoni120/*.tif'
    pathBuoniVal = '/Users/leonardoaggio/Desktop/Dataset/S606C/Buoni120Val/*.tif'
    contamination_N = 6/126.

pathFS = '/Users/leonardoaggio/Desktop/Dataset/S606C/FalsiScarti/*.tif'
pathScarti = '/Users/leonardoaggio/Desktop/Dataset/S606C/Scarti/*.tif'

pathFSVal = '/Users/leonardoaggio/Desktop/Dataset/S606C/FalsiScartiVal/*.tif'
pathScartiVal = '/Users/leonardoaggio/Desktop/Dataset/S606C/ScartiVal/*.tif'

pezziXimmagine = 1
border = 60
Nclusters = 10
nfeatures = 100
#
#


results = []
img_results = []
list_paths = []

list_paths += glob.glob(pathScarti)
results += [1] * len(glob.glob(pathScarti)) * pezziXimmagine

list_paths += glob.glob(pathBuoni) 
results += [0] * len(glob.glob(pathBuoni)) * pezziXimmagine

#list_paths += glob.glob(pathFS)
#results += [0] * len(glob.glob(pathFS)) * pezziXimmagine


#list_paths += glob.glob(pathBuoniVal) 
#results += [0] * len(glob.glob(pathBuoniVal)) * pezziXimmagine

#list_paths += glob.glob(pathFSVal)
#results += [0] * len(glob.glob(pathFSVal)) * pezziXimmagine

#list_paths += glob.glob(pathScartiVal)
#results += [1] * len(glob.glob(pathScartiVal)) * pezziXimmagine



#print(results)
i = 0
for img_path in list_paths:
    img = cv2.imread(img_path)
    temp = cutWorkSpace(img,border)
    #cv2.imwrite('ImgScarte/img'+str(i)+'.jpg', temp)
    temp2 = divideImage(temp,pezziXimmagine)
    img_results += temp2
    j = 0
    for im in temp2:
        #cv2.imwrite('ImgScarte/img'+str(i)+'pzs'+str(j)+'.jpg', im)
        j += 1
    i += 1

In [33]:
imgsWithClass = [img_results,results]


histo,kmeans = createNN(imgsWithClass, Nclusters)

10
Init 1/3 with method: k-means++
Inertia for init 1/3: 180339323.358154
Init 2/3 with method: k-means++
Inertia for init 2/3: 173836501.561324
Init 3/3 with method: k-means++
Inertia for init 3/3: 166728413.151355
Minibatch iteration 1/1300: mean batch inertia: 55117.904878, ewa inertia: 55117.904878 
Minibatch iteration 2/1300: mean batch inertia: 56927.621148, ewa inertia: 55413.864226 
Minibatch iteration 3/1300: mean batch inertia: 55169.216330, ewa inertia: 55373.854732 
Minibatch iteration 4/1300: mean batch inertia: 58089.223197, ewa inertia: 55817.923615 
Minibatch iteration 5/1300: mean batch inertia: 55758.895466, ewa inertia: 55808.270205 
Minibatch iteration 6/1300: mean batch inertia: 53526.375529, ewa inertia: 55435.091231 
Minibatch iteration 7/1300: mean batch inertia: 53516.786285, ewa inertia: 55121.373389 
Minibatch iteration 8/1300: mean batch inertia: 55752.348952, ewa inertia: 55224.562557 
Minibatch iteration 9/1300: mean batch inertia: 54626.930060, ewa inerti

In [34]:
import pandas as pd

print(Nclusters)
cols = []
hist_2 = []
i = 0
for h in histo:
    temp = []
    temp.append(list_paths[i])
    for el in h:
        temp.append(el)
    temp.append(results[i])
    i += 1
    hist_2.append(temp)
    
cols.append('name')
for i in range(Nclusters):
    cols.append('F'+str(i+1))

cols.append('R')

df = pd.DataFrame(hist_2 ,columns = cols)



10


In [35]:
histo[0]

array([0.  , 0.22, 0.02, 0.07, 0.01, 0.66, 0.02, 0.  , 0.  , 0.  ])

In [36]:
df

,name,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,R
0,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.000000,0.220000,0.020000,0.070000,0.01000,0.660000,0.020000,0.000000,0.000000,0.000000,1
1,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.020000,0.010000,0.050000,0.140000,0.07000,0.140000,0.380000,0.050000,0.060000,0.080000,1
2,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.030000,0.060000,0.050000,0.040000,0.15000,0.130000,0.340000,0.140000,0.030000,0.030000,1
3,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.029703,0.029703,0.019802,0.118812,0.19802,0.158416,0.336634,0.019802,0.009901,0.079208,1
4,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.250000,0.030000,0.150000,0.020000,0.32000,0.070000,0.120000,0.010000,0.010000,0.020000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
119,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.060000,0.120000,0.070000,0.040000,0.10000,0.150000,0.120000,0.170000,0.150000,0.020000,0
120,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.030000,0.050000,0.050000,0.080000,0.18000,0.070000,0.090000,0.260000,0.170000,0.020000,0
121,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.010000,0.070000,0.020000,0.160000,0.09000,0.510000,0.000000,0.010000,0.010000,0.120000,0
122,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.050000,0.010000,0.070000,0.090000,0.34000,0.040000,0.030000,0.200000,0.150000,0.020000,0


In [37]:
#sns.boxplot(histo[0])

In [38]:
random_state = np.random.RandomState(42)

In [39]:
col = []

for i in range(Nclusters):
    col.append('F'+str(i+1))

model=IsolationForest(n_estimators=100,max_samples='auto',contamination=contamination_N,random_state=random_state)

model.fit(df[col])

print(model.get_params())

{'bootstrap': False, 'contamination': 0.047619047619047616, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 100, 'n_jobs': None, 'random_state': RandomState(MT19937) at 0x7FD023688E20, 'verbose': 0, 'warm_start': False}


In [40]:
df['scores'] = model.decision_function(df[col])

df['anomaly_score'] = model.predict(df[col])

df[df['anomaly_score']==-1]

,name,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,R,scores,anomaly_score
0,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.00,0.22,0.02,0.07,0.01,0.66,0.02,0.00,0.00,0.00,1,-0.068179,-1
4,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.25,0.03,0.15,0.02,0.32,0.07,0.12,0.01,0.01,0.02,1,-0.105064,-1
21,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.05,0.03,0.03,0.14,0.00,0.06,0.33,0.10,0.05,0.21,0,-0.044544,-1
26,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.00,0.02,0.04,0.02,0.65,0.06,0.00,0.06,0.09,0.06,0,-0.010362,-1
42,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.04,0.08,0.01,0.17,0.04,0.43,0.02,0.00,0.00,0.21,0,-0.048166,-1
88,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.12,0.08,0.21,0.08,0.21,0.18,0.01,0.03,0.03,0.05,0,-0.021224,-1


In [41]:
df

,name,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,R,scores,anomaly_score
0,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.000000,0.220000,0.020000,0.070000,0.01000,0.660000,0.020000,0.000000,0.000000,0.000000,1,-0.068179,-1
1,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.020000,0.010000,0.050000,0.140000,0.07000,0.140000,0.380000,0.050000,0.060000,0.080000,1,0.042114,1
2,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.030000,0.060000,0.050000,0.040000,0.15000,0.130000,0.340000,0.140000,0.030000,0.030000,1,0.074225,1
3,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.029703,0.029703,0.019802,0.118812,0.19802,0.158416,0.336634,0.019802,0.009901,0.079208,1,0.039762,1
4,/Users/leonardoaggio/Desktop/Dataset/S606C/Sca...,0.250000,0.030000,0.150000,0.020000,0.32000,0.070000,0.120000,0.010000,0.010000,0.020000,1,-0.105064,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.060000,0.120000,0.070000,0.040000,0.10000,0.150000,0.120000,0.170000,0.150000,0.020000,0,0.080726,1
120,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.030000,0.050000,0.050000,0.080000,0.18000,0.070000,0.090000,0.260000,0.170000,0.020000,0,0.106039,1
121,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.010000,0.070000,0.020000,0.160000,0.09000,0.510000,0.000000,0.010000,0.010000,0.120000,0,0.013032,1
122,/Users/leonardoaggio/Desktop/Dataset/S606C/Buo...,0.050000,0.010000,0.070000,0.090000,0.34000,0.040000,0.030000,0.200000,0.150000,0.020000,0,0.114172,1


In [50]:
#temp = '/Users/leonardoaggio/Desktop/Dataset/S606C/Cam3@00004/*.tif'
temp = pathScarti
i = 0
test_path = glob.glob(temp) 
for img_path in test_path:
    img = cv2.imread(img_path)
    res = testNN(img,border,pezziXimmagine,Nclusters)
    if res[0] != -1:
        i += 1
    print(res)
print('TRAIN scarti')
print('BUONI : '+ str(i))
print('Falsi buoni su scarti totali: '+ str(i/len(test_path)))



[array([-1])]
[array([1])]
[array([1])]
[array([1])]
[array([-1])]
[array([1])]
TRAIN scarti
BUONI : 4
Falsi buoni su scarti totali: 0.6666666666666666


[]

In [54]:
temp = pathScartiVal
i = 0
test_path = glob.glob(temp) 
for img_path in test_path:
    img = cv2.imread(img_path)
    res = testNN(img,border,pezziXimmagine,Nclusters)
    if res[0] != -1:
        i += 1
    print(res)
print('Test scarti')
print('BUONI : '+ str(i))
print('Falsi buoni su scarti totali: '+ str(i/len(test_path)))

[<KeyPoint 0x7fd01f341120>, <KeyPoint 0x7fd023662c00>, <KeyPoint 0x7fd0235cb0c0>, <KeyPoint 0x7fd0235cbf30>, <KeyPoint 0x7fd0235cb7b0>, <KeyPoint 0x7fd0235cb270>, <KeyPoint 0x7fd0235cb840>, <KeyPoint 0x7fd0235cbc00>, <KeyPoint 0x7fd0235cbe70>, <KeyPoint 0x7fd0235cbdb0>, <KeyPoint 0x7fd0235cb720>, <KeyPoint 0x7fd0235cb990>, <KeyPoint 0x7fd0235cbc30>, <KeyPoint 0x7fd0235cb1b0>, <KeyPoint 0x7fd0235cb6c0>, <KeyPoint 0x7fd0235cbcf0>, <KeyPoint 0x7fd0235cb5a0>, <KeyPoint 0x7fd0235cb7e0>, <KeyPoint 0x7fd0235cb600>, <KeyPoint 0x7fd0235cb630>, <KeyPoint 0x7fd008469c00>, <KeyPoint 0x7fd008469030>, <KeyPoint 0x7fd008469510>, <KeyPoint 0x7fd008469270>, <KeyPoint 0x7fd0084698d0>, <KeyPoint 0x7fd008469ba0>, <KeyPoint 0x7fd008469480>, <KeyPoint 0x7fd008469990>, <KeyPoint 0x7fd008469630>, <KeyPoint 0x7fd008469c90>, <KeyPoint 0x7fd008469db0>, <KeyPoint 0x7fd008469210>, <KeyPoint 0x7fd008469f30>, <KeyPoint 0x7fd008469360>, <KeyPoint 0x7fd008469150>, <KeyPoint 0x7fd008469900>, <KeyPoint 0x7fd0084696c0>, 

In [44]:
#temp = '/Users/leonardoaggio/Desktop/Dataset/S606C/ScartiTotali/*.tif'
temp = pathBuoniVal
#temp = pathScarti
i = 0
j = 0
test_path = glob.glob(temp) 
for img_path in test_path:
    img = cv2.imread(img_path)
    res = testNN(img,border,pezziXimmagine,Nclusters)
    if res[0] != 1:
        i += 1
    if j % 10 == 0:
        print(str(j)+'/'+str(len(test_path)))
    
    j += 1
print('Test')
print('Scarti : '+ str(i))
print('Falsi scarti su buoni totali: '+ str(i/len(test_path)))

0/79
10/79
20/79
30/79
40/79
50/79
60/79
70/79
Test
Scarti : 4
Falsi scarti su buoni totali: 0.05063291139240506


In [45]:
temp = '/Users/leonardoaggio/Desktop/Dataset/S606C/Cam3@00004/*.tif'


i = 0
j = 0
test_path = glob.glob(temp) 
for img_path in test_path:
    img = cv2.imread(img_path)
    res = testNN(img,border,pezziXimmagine,Nclusters)
    if res[0] != 1:
        i += 1
    if j % 10 == 0:
        print(str(j)+'/'+str(len(test_path)))
    
    j += 1
print('Test')
print('Scarti : '+ str(i))
print('Falsi scarti su buoni totali: '+ str(i/len(test_path)))

0/437
10/437
20/437
30/437
40/437
50/437
60/437
70/437
80/437
90/437
100/437
110/437
120/437
130/437
140/437
150/437
160/437
170/437
180/437
190/437
200/437
210/437
220/437
230/437
240/437
250/437
260/437
270/437
280/437
290/437
300/437
310/437
320/437
330/437
340/437
350/437
360/437
370/437
380/437
390/437
400/437
410/437
420/437
430/437
Test
Scarti : 60
Falsi scarti su buoni totali: 0.13729977116704806
